In [3]:
from keras.models import load_model
import cv2
import numpy as np
from random import choice

In [4]:
REV_CLASS_MAP = {
    1:"rock",
    2:"paper",
    0:"scissors",
    3:"lizard",
    4:"spock",
    5:"none",
}

In [5]:
def mapper(val):
    return REV_CLASS_MAP[val]

In [6]:
REV_CLASS_MAP[1]

'rock'

In [7]:
def calculate_winner(move1, move2):
    if move1 == move2:
        return "Tie"

    if move1 == "rock":
        if move2 == "scissors":
            return "User"
        if move2 == "paper":
            return "Computer"
        if move2 == "lizard":
            return "User"
        if move2 == "spock":
            return "Computer"

    if move1 == "paper":
        if move2 == "rock":
            return "User"
        if move2 == "scissors":
            return "Computer"
        if move2 == "lizard":
            return "Computer"
        if move2 == "spock":
            return "User"
        

    if move1 == "scissors":
        if move2 == "paper":
            return "User"
        if move2 == "rock":
            return "Computer"
        if move2 == "lizard":
            return "User"
        if move2 == "spock":
            return "Computer"
        
    if move1 == "lizard":
        if move2 == "paper":
            return "User"
        if move2 == "rock":
            return "Computer"
        if move2 == "scissors":
            return "Computer"
        if move2 == "spock":
            return "User"
        
    if move1 == "spock":
        if move2 == "paper":
            return "Computer"
        if move2 == "rock":
            return "Computer"
        if move2 == "scissors":
            return "User"
        if move2 == "lizard":
            return "User"
        
        

In [8]:
model = load_model("rock_paper_scissors_lizrd_spock_cnn.h5")

In [15]:

cap = cv2.VideoCapture(0)

prev_move = None

while True:
    ret, frame = cap.read()
    if not ret:
        continue

    # rectangle for user to play
    cv2.rectangle(frame, (0, 0), (250, 250), (255, 255, 255), 2)
    # rectangle for computer to play
    cv2.rectangle(frame, (350, 0), (550, 250), (255, 255, 255), 2)

    # extract the region of image within the user rectangle
    roi = frame[50:250, 50:250]
    roi = roi/255.0
#     img = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
#     img = cv2.resize(img, (227, 227))

    # predict the move made
    pred = model.predict(np.array([roi]))
    move_code = np.argmax(pred[0])
#     print(move_code)
    user_move_name = mapper(move_code)

    # predict the winner (human vs computer)
    if prev_move != user_move_name:
        if user_move_name != "none":
            computer_move_name = choice(['rock', 'paper', 'scissors', 'lizard', 'spock'])
            winner = calculate_winner(user_move_name, computer_move_name)
        else:
            computer_move_name = "none"
            winner = "Waiting..."
    prev_move = user_move_name

    # display the information
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(frame, "Your Move: " + user_move_name,
                (50, 275), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Computer's Move: " + computer_move_name,
                (250, 275), font, 0.6, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.putText(frame, "Winner: " + winner,
                (300, 300), font, 0.8, (0, 0, 255), 4, cv2.LINE_AA)

    if computer_move_name != "none":
        icon = cv2.imread(
            "images/{}.png".format(computer_move_name))
        icon = cv2.resize(icon, (200, 200))
        frame[0:200, 350:550] = icon

    cv2.imshow("Rock Paper Scissors", frame)

    k = cv2.waitKey(10)
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
